这个笔记本使用原生 Python 绑定，为 MuJoCo 物理提供了一个入门教程
摘自：https://github.com/google-deepmind/mujoco/blob/main/python/tutorial.ipynb

# All imports

In [1]:
# !pip install mujoco

# Set up GPU rendering.
# from google.colab import files
import distutils.util
import os
import subprocess
if subprocess.run('nvidia-smi').returncode:
    raise RuntimeError(
        'Cannot communicate with GPU. '
        'Make sure you are using a GPU Colab runtime. '
        'Go to the Runtime menu and select Choose runtime type.')

# Add an ICD config so that glvnd can pick up the Nvidia EGL driver.
# This is usually installed as part of an Nvidia driver package, but the Colab
# kernel doesn't install its driver via APT, and as a result the ICD is missing.
# (https://github.com/NVIDIA/libglvnd/blob/master/src/EGL/icd_enumeration.md)
NVIDIA_ICD_CONFIG_PATH = '/usr/share/glvnd/egl_vendor.d/10_nvidia.json'
if not os.path.exists(NVIDIA_ICD_CONFIG_PATH):
    with open(NVIDIA_ICD_CONFIG_PATH, 'w') as f:
        f.write("""{
    "file_format_version" : "1.0.0",
    "ICD" : {
        "library_path" : "libEGL_nvidia.so.0"
    }
}
""")

# Configure MuJoCo to use the EGL rendering backend (requires GPU)
print('Setting environment variable to use GPU rendering:')
%env MUJOCO_GL=egl

# Check if installation was succesful.
try:
    print('Checking that the installation succeeded:')
    import mujoco
    mujoco.MjModel.from_xml_string('<mujoco/>')
except Exception as e:
    raise e from RuntimeError(
        'Something went wrong during installation. Check the shell output above '
        'for more information.\n'
        'If using a hosted Colab runtime, make sure you enable GPU acceleration '
        'by going to the Runtime menu and selecting "Choose runtime type".')

print('Installation successful.')

# Other imports and helper functions
import time
import itertools
import numpy as np

# Graphics and plotting.
print('Installing mediapy:')
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
!pip install -q mediapy
import mediapy as media
import matplotlib.pyplot as plt

# More legible printing from numpy.
np.set_printoptions(precision=3, suppress=True, linewidth=100)

from IPython.display import clear_output
clear_output()


# MuJoCo 基础

我们首先定义并加载一个简单的模型：

In [2]:
xml = """
<mujoco>
  <worldbody>
    <geom name="red_box" type="box" size=".2 .2 .2" rgba="1 0 0 1"/>
    <geom name="green_sphere" pos=".2 .2 .2" size=".1" rgba="0 1 0 1"/>
  </worldbody>
</mujoco>
"""
model = mujoco.MjModel.from_xml_string(xml)
# from_xml_string:从XML字符串和可选的资源字典中加载MjModel。
# mujoco.MjModel 类没有 Python 构造函数。相反，我们提供了三个静态工厂函数来创建新的 mjModel 实例： mujoco.MjModel.from_xml_string 、mujoco.MjModel.from_xml_path，和 mujoco.MjModel.from_binary_path 。第一个函数接受一个模型 XML 作为字符串，而后两个函数接受 XML 或 MJB 模型文件的路径。这三个函数都可选地接受一个 Python 字典，该字典会被转换为 MuJoCo 虚拟文件系统 ，用于模型编译过程中。
print(model)
print(type(model))

<class 'mujoco._structs.MjModel'>


## mjModel

In [3]:
model.ngeom  # number of geoms 几何体的数量

2

In [4]:
model.geom_rgba  # rgba when material is omitted (ngeom x 4)

array([[1., 0., 0., 1.],
       [0., 1., 0., 1.]], dtype=float32)

## Named access 命名访问

In [5]:
try:
    model.geom()
except KeyError as e:
    print(e)
    print(type(e))

"Invalid name ''. Valid names: ['green_sphere', 'red_box']"
<class 'KeyError'>


In [6]:
model.geom('green_sphere')
model.geom('red_box')

<_MjModelGeomViews
  bodyid: array([0], dtype=int32)
  conaffinity: array([1], dtype=int32)
  condim: array([3], dtype=int32)
  contype: array([1], dtype=int32)
  dataid: array([-1], dtype=int32)
  friction: array([1.   , 0.005, 0.   ])
  gap: array([0.])
  group: array([0], dtype=int32)
  id: 0
  margin: array([0.])
  matid: array([-1], dtype=int32)
  name: 'red_box'
  pos: array([0., 0., 0.])
  priority: array([0], dtype=int32)
  quat: array([1., 0., 0., 0.])
  rbound: array([0.346])
  rgba: array([1., 0., 0., 1.], dtype=float32)
  sameframe: array([1], dtype=uint8)
  size: array([0.2, 0.2, 0.2])
  solimp: array([0.9  , 0.95 , 0.001, 0.5  , 2.   ])
  solmix: array([1.])
  solref: array([0.02, 1.  ])
  type: array([6], dtype=int32)
  user: array([], dtype=float64)
>

In [7]:
model.geom('green_sphere').rgba

array([0., 1., 0., 1.], dtype=float32)

In [8]:
id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_GEOM, 'green_sphere')
model.geom_rgba[id, :]

array([0., 1., 0., 1.], dtype=float32)

In [9]:
print('id of "green_sphere": ', model.geom('green_sphere').id)
print('name of geom 1: ', model.geom(1).name)
print('name of body 0: ', model.body(0).name)

id of "green_sphere":  1
name of geom 1:  green_sphere
name of body 0:  world


In [10]:
[model.geom(i).name for i in range(model.ngeom)]

['red_box', 'green_sphere']

## `mjData`

In [11]:
data = mujoco.MjData(model)

In [12]:
print(data.geom_xpos)  # Cartesian geom position (ngeom x 3)

[[0. 0. 0.]
 [0. 0. 0.]]


In [13]:
mujoco.mj_kinematics(model, data)
print('raw access:\n', data.geom_xpos)

# MjData also supports named access:
print('\nnamed access:\n', data.geom('green_sphere').xpos)
geom_green_sphere = data.geom('green_sphere')
print('\nnamed access:\n', geom_green_sphere)

raw access:
 [[0.  0.  0. ]
 [0.2 0.2 0.2]]

named access:
 [0.2 0.2 0.2]

named access:
 <_MjDataGeomViews
  id: 1
  name: 'green_sphere'
  xmat: array([1., 0., 0., 0., 1., 0., 0., 0., 1.])
  xpos: array([0.2, 0.2, 0.2])
>
